<div style="display: flex; background-color: RGB(255,114,0);" >
<h1 style="margin: auto; padding: 30px; ">Brief 9 -  </h1>
</div>

<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Partie 1 - Data Preprocessing</h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">1.1 - Importation des librairies</h3>
</div>

In [56]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from pathlib import Path
from clean.Cleaning import data_check
from clean.Cleaning import record
from clean.Cleaning import missing_values
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import pickle
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">1.2 - Déclaration des fonctions</h3>
</div>

In [57]:
def check_unique(cat):
    cat_val = data[cat].unique()
    print('{} ({}):'.format(cat,len(cat_val)))
    print('{}\n'.format(cat_val))

<div style="background-color: RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color:#fff; ">1.3 - Importation et préparation des fichiers</h3>
</div>

In [58]:
data = pd.read_csv(r'../data/RAW/valeursfoncieres-2020.txt', encoding="UTF8", sep='|')
df = data.copy()

In [4]:
missing_values(df).sort_values('Manquant %', ascending=False)

Nombre d'observations: 2,459,560
Nombre de valeurs: 105,761,080
Valeurs manquantes: 60,808,308
Qualité des données: 42.5%
Type de données:
 object     22
float64    17
int64       4
dtype: int64%


,Manquant,Manquant %,Type
Code service CH,2459560,100.00,float64
4 Articles CGI,2459560,100.00,float64
Reference document,2459560,100.00,float64
5 Articles CGI,2459560,100.00,float64
Identifiant local,2459560,100.00,float64
3 Articles CGI,2459560,100.00,float64
2 Articles CGI,2459560,100.00,float64
1 Articles CGI,2459560,100.00,float64
Surface Carrez du 5eme lot,2458997,99.98,object
Surface Carrez du 4eme lot,2458303,99.95,object


In [ ]:
plt.figure(figsize=(20,6))
sns.heatmap(df.isnull(), yticklabels = False, cbar = False )

In [ ]:
data_check(df)

<div style="color: RGB(255,114,0)">
<h3>Information Vigilance</h3>
Pour ce premier dataset, nous n'observons plusieurs anomalies.<br/>
Des colonnes totalement vides et 75939 ligne en doublons.

In [59]:
df = df.drop(['Code service CH', 'Reference document', '1 Articles CGI', '2 Articles CGI',
              '3 Articles CGI', '4 Articles CGI', '5 Articles CGI', 'Identifiant local'],axis= 1)

In [ ]:
liste=['No disposition', 'Nature mutation', 'No voie', 'B/T/Q', 'Type de voie',  'Code postal',
       'Code departement', 'Code commune', 'Section','Type local', 'Nombre pieces principales', 'Nature culture']
for cat in liste:
    check_unique(cat)

In [60]:
#df['Date mutation'] = df['Date mutation'].transform(lambda x: datetime.strptime(x, '%d/%m/%Y'))
df['Valeur fonciere'] = df['Valeur fonciere'].str.replace(',','.')
df.loc[df['B/T/Q'].isin([',','/','*',"'",'-','.']), ['B/T/Q']] = np.NaN

In [61]:
df['Nature culture'] = df['Nature culture'].fillna("None")
df['Valeur fonciere'] = df['Valeur fonciere'].fillna(0)
df['No voie'] = df['No voie'].fillna(0)
df['Type de voie'] = df['Type de voie'] .fillna('')
df['B/T/Q'] = df['B/T/Q'].fillna('')
df['Section'] = df['Section'].fillna('')
df['Nombre pieces principales'] = df['Nombre pieces principales'].fillna(0)
df['Code postal'] = df['Code postal'].fillna(0)

In [62]:
# Colonne Code Département
df['Code departement'] = df['Code departement'].astype('str')
df['Code departement'] = df['Code departement'].str.replace('2A','120')
df['Code departement'] = df['Code departement'].str.replace('2B','121')

In [63]:
# convertir les types de variables
df['Code departement'] = df['Code departement'].astype('int')
df['No voie'] = df['No voie'].astype('int')
df['Nombre pieces principales'] = df['Nombre pieces principales'].astype('int')
df['Code postal'] = df['Code postal'].astype('int')
df['Valeur fonciere'] = df['Valeur fonciere'].astype('float')

In [64]:
df['Code postal'] = df['Code postal'].apply(lambda x: '0{}'.format(x) if len(str(x))== 4 else x)
df['Cadastre'] = df['Code departement'].apply(str) + ' ' +  df['Code commune'].apply(str) + ' ' + df['Section']
df['Adresse'] = df['No voie'].apply(str) + ' '+ df['B/T/Q'] + ' ' + df['Type de voie'] + ' ' +  df['Voie'] + ' ' + df['Code postal'].apply(str) + ' ' + df['Commune']
df['Adresse'] = df['Adresse'].fillna('')

In [65]:
df = df.drop(['No voie','B/T/Q','Type de voie','Voie','Code postal','3eme lot','Surface Carrez du 5eme lot',
             'Section','No Volume','Prefixe de section','Nature culture speciale','Code voie',
              'Surface Carrez du 4eme lot','5eme lot','Surface Carrez du 3eme lot','4eme lot','Code type local',
              'Surface Carrez du 2eme lot','2eme lot','Surface Carrez du 1er lot', '1er lot',
              'Code commune' ],axis= 1)

In [66]:
df.drop(df[(df['Type local'].isna()) & (df['Surface reelle bati'].isna()) & (df['Surface terrain'].isna())].index, inplace=True)

In [67]:
df.loc[(df[df['Type local'].isna() & (df['Surface terrain'].notna()) 
                 & (df['Surface reelle bati'].isna())]).index, 'Type local'] = 'Terrain'
df.loc[df[df['Type local']=='Terrain'].index, 'Surface reelle bati'] = 0

In [68]:
df.loc[(df[(df['Surface reelle bati']!=0) & 
                 (df['Surface terrain'].isna())]).index, 'Surface terrain'] = 0
df.loc[(df[(df['Surface reelle bati']==0) & 
                 (df['Surface terrain'].isna())]).index, 'Surface terrain'] = 0

In [69]:
df.loc[(df[df['Type local']=='Terrain']).index, ['Nombre pieces principales']] = 0
df.loc[(df[df['Type local']=='Local industriel. commercial ou assimilé']).index, ['Nombre pieces principales']] = 0

In [70]:
df.loc[(df[df['Nature culture']=='None']).index, ['Surface terrain']] = 0

In [71]:
df['Surface reelle bati'] = df['Surface reelle bati'].fillna(0)

In [72]:
missing_values(df).sort_values('Manquant %', ascending=False)

Nombre d'observations: 2,287,657
Nombre de valeurs: 34,314,855
Valeurs manquantes: 0
Qualité des données: 100.0%
Type de données:
 object     7
float64    3
int64      3
int32      2
dtype: int64%


,Manquant,Manquant %,Type
No disposition,0,0.0,int64
Date mutation,0,0.0,object
Nature mutation,0,0.0,object
Valeur fonciere,0,0.0,float64
Commune,0,0.0,object
Code departement,0,0.0,int32
No plan,0,0.0,int64
Nombre de lots,0,0.0,int64
Type local,0,0.0,object
Surface reelle bati,0,0.0,float64


In [73]:
df['Id'] = df['Nature mutation'] + ' ' +  df['Date mutation'].apply(str)+ ' ' + df['Cadastre'] + ' '+ df['Valeur fonciere'].apply(str)

In [76]:
df=df[df['Nature mutation']=='Vente']

In [80]:
record(df,'df_clean','CURATED')

<div style="color: RGB(255,114,0)">
<h3>Information Vigilance</h3>
la colonne 'Type local' contient 45% des valeurs None.<br/>
0 Valeur pour la colonne 'surface reel bati' donc on considère que c'est du terrain.